In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
from torchvision import transforms
from PIL import Image 

def load_and_preprocess_img(
    path, new_size, crop, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
):
    img = Image.open(path)
    compose = transforms.Compose(
        [
            transforms.Resize(new_size),
            transforms.CenterCrop(crop),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ]
    )
    img = compose(img)
    img = img.unsqueeze(0).permute((0, 2, 3, 1))
    return img.numpy()

In [19]:
import ivy
ivy.set_backend("torch")

# Load image
this_dir = "/models/images/cat.jpg" 
img = ivy.asarray(load_and_preprocess_img(this_dir, 256, 224))

test

In [20]:
# conv = ivy.Conv2D(3, 128, [1,1], 5, 4)
# display(img.shape)

# output = conv(img)
# display(output.shape)

# conv = ivy.Conv2D(128 , 256, [1,1], 5, 4)
# output = conv(output)
# display(output.shape)

procedural model execution for debugging start

In [21]:
# ConvBlock

def ConvBlock(in_chaivyels, out_chaivyels, kernel_size, stride, padding, x):
    conv = ivy.Conv2D(in_chaivyels, out_chaivyels, kernel_size, stride, padding)
    bn = ivy.BatchNorm2D(out_chaivyels)
    activation = ivy.ReLU()

    x = conv(x)
    x = bn(x)
    x = activation(x)
    return x

In [22]:
# _ConvBlock

class _ConvBlock(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(_ConvBlock, self).__init__()

        self.conv = ivy.Conv2D(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = ivy.BatchNorm2D(out_channels)
        self.activation = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [23]:
# Inception Block

def InceptionBlock(in_channels, num1x1, num3x3_reduce, num3x3, num5x5_reduce, num5x5, pool_proj, x):
    block1 = ivy.Sequential(_ConvBlock(in_channels, num1x1, kernel_size=[1, 1], stride=1, padding=0))

    block2 = ivy.Sequential(_ConvBlock(in_channels, num3x3_reduce, kernel_size=[1, 1], stride=1, padding=0),
                            _ConvBlock(num3x3_reduce, num3x3, kernel_size=[3, 3], stride=1, padding=1))
    
    block3 = ivy.Sequential(_ConvBlock(in_channels, num5x5_reduce, kernel_size=[1, 1], stride=1, padding=0),
                            _ConvBlock(num5x5_reduce, num5x5, kernel_size=[3, 3], stride=1, padding=1))
    
    block4 = ivy.Sequential(ivy.MaxPool2D(3, 1, 1),
                            _ConvBlock(in_channels, pool_proj, kernel_size=[1, 1], stride=1, padding=0))

    block1 = block1(x)
    block2 = block2(x)
    block3 = block3(x)
    block4 = block4(x)
    
    return ivy.concat([block1, block2, block3, block4], axis=3)

In [24]:
# Auxiliary Block

def AuxiliaryBlock(in_channels, num_classes, x):
    pool = ivy.AdaptiveAvgPool2d((4, 4))
    conv = ivy.Conv2D(in_channels, 128, [1,1], 1, 0)
    activation = ivy.ReLU()
    fc1 = ivy.Linear(2048, 1024)
    dropout = ivy.Dropout(0.7)
    fc2 = ivy.Linear(1024, num_classes)

    
    out = pool(x)
    display(out.shape)
    out = conv(out)
    display(out.shape)
    out = activation(out)
    display(out.shape)
    out = ivy.flatten(out, start_dim=1)(out)
    display(out.shape)
    out = fc1(out)
    display(out.shape)
    out = activation(out)
    display(out.shape)
    out = dropout(out)
    display(out.shape)
    out = fc2(out)
    display(out.shape)
    return out

In [25]:
# MaxPool2D

def MaxPool2D(kernel_size, stride, padding, x):
    pool = ivy.MaxPool2D(kernel_size, stride, padding)
    x = pool(x)
    return x

In [26]:
# AdaptiveAvgPool2D

def AdaptiveAvgPool2D(output_size, x):
    avg_pool = ivy.AdaptiveAvgPool2d(output_size)
    x = avg_pool(x)
    return x

In [27]:
# Dropout

def Dropout(prob, x):
    # prob: The probability of zeroing out each array element.
    dropout = ivy.Dropout(prob)
    x = dropout(x)
    return x

In [28]:
# FC

def FC(input_channels, output_channels, x):
    linear = ivy.Linear(input_channels, output_channels)
    x = linear(x)
    return x

In [29]:
# # ivy.flatten(out, start_dim=1)

# def ivy.flatten(out, start_dim=1)(x):
#     x = ivy.flatten(out, start_dim=1)(x)
#     return x

In [30]:
# GoogleNet V1

num_classes = 1000
display(img.shape)
out = ConvBlock(3, 64, [7,7], 2, 3, img)
display(out.shape)

ivy.Shape(1, 224, 224, 3)

ivy.Shape(1, 112, 112, 64)

In [31]:
out = MaxPool2D([3,3], 2, 1, out)
display(out.shape)


ivy.Shape(1, 56, 56, 64)

In [32]:
out = ConvBlock(64, 64, [1,1], 1, 0, out)
display(out.shape)


ivy.Shape(1, 56, 56, 64)

In [33]:
out = ConvBlock(64, 192, [3,3], 1, 1, out)
display(out.shape)


ivy.Shape(1, 56, 56, 192)

In [34]:
out = MaxPool2D(3, 2, 1, out)
display(out.shape)


ivy.Shape(1, 28, 28, 192)

In [35]:
out = InceptionBlock(192, 64, 96, 128, 16, 32, 32, out)
display(out.shape)


ivy.Shape(1, 28, 28, 256)

In [36]:
out = InceptionBlock(256, 128, 128, 192, 32, 96, 64, out)
display(out.shape)


ivy.Shape(1, 28, 28, 480)

In [37]:
out = MaxPool2D(3, 2, 1, out)
display(out.shape)


ivy.Shape(1, 14, 14, 480)

In [38]:
out_1 = InceptionBlock(480, 192, 96, 208, 16, 48, 64, out)
display(out_1.shape)

ivy.Shape(1, 14, 14, 512)

In [39]:
in_channels1=512
num_classes=1000
pool = ivy.AvgPool2D((5, 5), 3, 0) # ivy.Shape(1, 4, 4, 512)
conv = ivy.Conv2D(in_channels1, 128, [1,1], 1, 0)
activation = ivy.ReLU()
fc1 = ivy.Linear(2048, 1024)
dropout = ivy.Dropout(0.7)
fc2 = ivy.Linear(1024, num_classes)

display(out_1.shape)
out_3 = pool(out_1)
display(out_3.shape)
out = conv(out_3)
out = activation(out)
out = ivy.flatten(out, start_dim=1)
display(out.shape)
out = fc1(out)
display(out.shape)
out = activation(out)
display(out.shape)
out = dropout(out)
display(out.shape)
out = fc2(out)
display(out.shape)
aux1 = out
display(aux1.shape)

ivy.Shape(1, 14, 14, 512)

ivy.Shape(1, 4, 4, 512)

ivy.Shape(1, 2048)

ivy.Shape(1, 1024)

ivy.Shape(1, 1024)

ivy.Shape(1, 1024)

ivy.Shape(1, 1000)

ivy.Shape(1, 1000)

In [40]:
out = InceptionBlock(512, 160, 112, 224, 24, 64, 64, out_1)
display(out.shape)

ivy.Shape(1, 14, 14, 512)

In [41]:
out = InceptionBlock(512, 128, 128, 256, 24, 64, 64, out)
display(out.shape)
out_5 = InceptionBlock(512, 112, 144, 288, 32, 64, 64, out)
display(out_5.shape)

ivy.Shape(1, 14, 14, 512)

In [ ]:
in_channels1=528
num_classes=1000
pool = ivy.AvgPool2D((5, 5), 3, 0)
conv = ivy.Conv2D(in_channels1, 128, [1,1], 1, 0)
activation = ivy.ReLU()
fc1 = ivy.Linear(2048, 1024)
dropout = ivy.Dropout(0.7)
fc2 = ivy.Linear(1024, num_classes)

display(out_5.shape)
out_3 = pool(out_5)
display(out_3.shape)
out = conv(out_3)
out = activation(out)
out = ivy.flatten(out, start_dim=1)
display(out.shape)
out = fc1(out)
display(out.shape)
out = activation(out)
display(out.shape)
out = dropout(out)
display(out.shape)
out = fc2(out)
display(out.shape)
aux2 = out
display(aux2.shape)

ivy.Shape(1, 14, 14, 528)

ivy.Shape(1, 4, 4, 528)

ivy.Shape(1, 2048)

ivy.Shape(1, 1024)

ivy.Shape(1, 1024)

ivy.Shape(1, 1024)

ivy.Shape(1, 1000)

ivy.Shape(1, 1000)

In [ ]:
out = InceptionBlock(528, 256, 160, 320, 32, 128, 128, out_5)
display(out.shape)
out_6 = MaxPool2D(3, 2, 1, out)
display(out_6.shape)

ivy.Shape(1, 14, 14, 832)

ivy.Shape(1, 7, 7, 832)

In [ ]:
out = InceptionBlock(832, 256, 160, 320, 32, 128, 128, out_6)
display(out.shape)
out_7 = InceptionBlock(832, 384, 192, 384, 48, 128,128, out)
display(out_7.shape)


ivy.Shape(1, 7, 7, 832)

ivy.Shape(1, 7, 7, 1024)

In [ ]:
pool7 = ivy.AvgPool2D((7,7), 1, 0)
out = pool7(out_7)
display(out.shape)

out = ivy.flatten(out, start_dim=1)
display(out.shape)
display(out)
out_8 = Dropout(0.4, out)
display(out_8)

ivy.Shape(1, 1, 1, 1024)

ivy.Shape(1, 1024)

ivy.array([[4.60735750e+00, 9.01112671e+01, 3.61349317e-03, ...,
        1.14853615e+02, 1.26411003e+02, 6.81729126e+01]])

ivy.array([[7.67892885e+00, 1.50185440e+02, 6.02248870e-03, ...,
        0.00000000e+00, 2.10684998e+02, 1.13621521e+02]])

In [ ]:
out = FC(1024, num_classes, out_8)

display(out.shape, aux1.shape, aux2.shape)

ivy.Shape(1, 1000)

ivy.Shape(1, 1000)

ivy.Shape(1, 1000)

In [ ]:
import os
import ivy
import pytest
import numpy as np
import jax

# Enable x64 support in JAX
jax.config.update("jax_enable_x64", True)
from ivy_models_tests import helpers
from ivy_models.resnet import resnet_18, resnet_34, resnet_50, resnet_152, resnet_101


In [ ]:
def np_softmax(inputs):
    """apply the softmax on the output"""
    return np.exp(inputs) / np.sum(np.exp(inputs))


In [ ]:


import warnings
warnings.filterwarnings("ignore")

def test_resnet_18_img_classification():
    """Test ResNet-18 image classification."""
    num_classes = 1000

    # Load image
    img = ivy.asarray(
        helpers.load_and_preprocess_img(
            "/models/images/cat.jpg", 256, 224
        ),
    )

    # Create model
    model = resnet_18(pretrained=True)
    
    # Perform inference
    output = model(img)

    display(model.state_dict)
    # Cardinality test
    assert output.shape == tuple([ivy.to_scalar(True), num_classes])

    # Value test
    if True:
        output = output[0]
        true_indices = ivy.array([282, 281, 285])
        calc_indices = ivy.argsort(output, descending=True)[:3]

        assert np.array_equal(true_indices, calc_indices)

        true_logits = np.array([0.7069, 0.2663, 0.0231])
        calc_logits = np.take(
            np_softmax(ivy.to_numpy(output)), ivy.to_numpy(calc_indices)
        )

        assert np.allclose(true_logits, calc_logits, rtol=0.005)


test_resnet_18_img_classification()